# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_csv = pd.read_csv('../output_data/weather.csv')
weather_csv


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.12,-134.97,71.98,77,49,20.69,PF,1600789833
1,Busselton,-33.65,115.33,48.99,71,29,6.85,AU,1600789401
2,Khatanga,71.97,102.50,47.35,69,14,10.13,RU,1600789834
3,Hermanus,-34.42,19.23,64.99,63,0,3.00,ZA,1600789834
4,Jamestown,42.10,-79.24,61.00,62,1,5.17,US,1600789834
...,...,...,...,...,...,...,...,...,...
546,Maputo,-25.97,32.59,69.80,56,75,5.82,MZ,1600789975
547,Yatou,37.15,122.38,63.55,74,98,5.86,CN,1600789975
548,Forsytheganj,52.63,29.73,61.95,62,0,6.29,BY,1600789975
549,Leh,34.17,77.58,55.42,32,3,1.99,IN,1600789976


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)

In [5]:
# Store 'Lat' and 'Lng' into  locations 
weather_csv = weather_csv.dropna()

locations = weather_csv[["Lat", "Lng"]].astype(float)

#humidity = weather_csv[["Humidity"]].astype(float)
#There is a problem converting humidity to float, so using another approach

weather_csv['Humidity'] = pd.to_numeric(weather_csv['Humidity'], downcast='float')

print(weather_csv.dtypes)


City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity      float32
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object


C:\Users\anjan\anaconda3\envs\bcs\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [10]:
#Adding heat map layer
fig = gmaps.figure(map_type="HYBRID")

heat_layer = gmaps.heatmap_layer(locations, weights = weather_csv['Humidity'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
good_weather = weather_csv[(weather_csv['Max Temp']>70) & (weather_csv['Max Temp']<80)
                          & (weather_csv['Wind Speed'] < 10) & (weather_csv['Cloudiness']==0)]

good_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,Amsterdam,52.37,4.89,72.00,52.0,0,8.05,NL,1600789701
37,San Quintín,30.48,-115.95,75.51,59.0,0,7.23,MX,1600789803
262,Eišiškės,54.17,25.00,71.01,60.0,0,6.93,LT,1600789902
317,Alyangula,-13.85,136.42,71.60,88.0,0,4.16,AU,1600789916
330,Tūkrah,32.53,20.58,77.77,61.0,0,8.99,LY,1600789919
348,Nhulunbuy,-12.23,136.77,71.60,94.0,0,6.53,AU,1600789925
374,Karratha,-20.74,116.85,70.54,48.0,0,8.75,AU,1600789933
395,Gifhorn,52.48,10.55,78.80,31.0,0,6.93,DE,1600789938
417,Rotterdam,51.92,4.48,73.00,53.0,0,6.93,NL,1600789729
431,Torata,-17.08,-70.84,71.96,20.0,0,9.71,PE,1600789947


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
# Store into variable named hotel_df
hotel_df = good_weather.reset_index(drop=True)

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}


In [14]:
# Hit the Google Places API for each city's coordinates.
# use iterrows to iterate through pandas dataframe
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in hotel_df.iterrows():

    # get coordinates
    lat = row['Lat']
    lng = row['Lng']

    # add keyword to params dict
    params['location'] = f'{lat},{lng}'

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
              
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
       
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
   
    
hotel_df

------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Amsterdam,52.37,4.89,72.00,52.0,0,8.05,NL,1600789701,Ambassade Hotel
1,San Quintín,30.48,-115.95,75.51,59.0,0,7.23,MX,1600789803,La Cueva del Pirata
2,Eišiškės,54.17,25.00,71.01,60.0,0,6.93,LT,1600789902,Esperanza Resort & SPA
3,Alyangula,-13.85,136.42,71.60,88.0,0,4.16,AU,1600789916,Groote Eylandt Lodge
4,Tūkrah,32.53,20.58,77.77,61.0,0,8.99,LY,1600789919,Alnooran Villa Resort
5,Nhulunbuy,-12.23,136.77,71.60,94.0,0,6.53,AU,1600789925,Walkabout Lodge & Tavern
6,Karratha,-20.74,116.85,70.54,48.0,0,8.75,AU,1600789933,Samson Beach Chalets
7,Gifhorn,52.48,10.55,78.80,31.0,0,6.93,DE,1600789938,Wassermühle Eldingen
8,Rotterdam,51.92,4.48,73.00,53.0,0,6.93,NL,1600789729,the niu Fender
9,Torata,-17.08,-70.84,71.96,20.0,0,9.71,PE,1600789947,Hotel Moquegua


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display figure
fig




Figure(layout=FigureLayout(height='420px'))